In [18]:
import requests
import time
import pandas as pd

In [21]:
def get_puuid(region, summoner_name, api_key):
    api_url = (
        "https://" + 
        region +
        ".api.riotgames.com/lol/summoner/v4/summoners/by-name/" +
        summoner_name +
        "?api_key=" +
        api_key
    )
    
    print(api_url)
    
    resp = requests.get(api_url)
    player_info = resp.json()
    puuid = player_info['puuid']
    return puuid

#Query parameter 'count' must have a maximum value of 100

In [22]:
api_key = "RGAPI-ae5e7b51-b068-4f78-95b9-584136b49bc9"
region = "euw1"
summoner_name = "fallinglove"
puuid = get_puuid(region, summoner_name, api_key)
puuid

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/fallinglove?api_key=RGAPI-ae5e7b51-b068-4f78-95b9-584136b49bc9


'-3doM1YXNXH8dExYK3kq8vw06KShdkJ-hECzwo8seV1lkHujhDq2R1kzDggD6ZZHG6Xbs8URshdTaQ'

In [4]:
def get_match_ids(puuid, mass_region, no_games, queue_id, api_key):
    api_url = (
        "https://" +
        mass_region +
        ".api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + 
        "/ids?start=0" + 
        "&count=" +
        str(no_games) + 
        "&queue=" + 
        str(queue_id) + 
        "&api_key=" + 
        api_key
    )
    
    print(api_url)
    
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

In [ ]:
#Query parameter 'count' must have a maximum value of 100

In [7]:
mass_region = 'europe'
no_games = 10
queue_id = 420
match_ids = get_match_ids(puuid, mass_region, no_games, queue_id, api_key)
match_ids

https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/-3doM1YXNXH8dExYK3kq8vw06KShdkJ-hECzwo8seV1lkHujhDq2R1kzDggD6ZZHG6Xbs8URshdTaQ/ids?start=0&count=10&queue=420&api_key=RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02


['EUW1_6639695853',
 'EUW1_6639674317',
 'EUW1_6639653949',
 'EUW1_6639645836',
 'EUW1_6639597933',
 'EUW1_6639585296',
 'EUW1_6639577072',
 'EUW1_6636841353',
 'EUW1_6636834892',
 'EUW1_6636812567']

In [8]:
def get_match_data(match_id, mass_region, api_key):
    api_url = (
        "https://" + 
        mass_region + 
        ".api.riotgames.com/lol/match/v5/matches/" +
        match_id + 
        "?api_key=" + 
        api_key
    )
    
    # we need to add this "while" statement so that we continuously loop until it's successful
    while True:
        resp = requests.get(api_url)
        
        # whenever we see a 429, we sleep for 10 seconds and then restart from the top of the "while" loop
        if resp.status_code == 429:
            print("Rate Limit hit, sleeping for 10 seconds")
            time.sleep(10)
            # continue means start the loop again
            continue
            
        # if resp.status_code isn't 429, then we carry on to the end of the function and return the data
        match_data = resp.json()
        return match_data   

In [9]:
def find_player_data(match_data, puuid):
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    player_match_data = match_data['info']['participants'][player_index]
    return player_match_data

In [10]:
def gather_all_player_match_data(puuid, match_ids, mass_region, api_key):
    player_data_list = []

    for match_id in match_ids:
        print(match_id)
    
        match_data = get_match_data(match_id, mass_region, api_key)
        player_match_data = find_player_data(match_data, puuid)
        player_data_list.append(player_match_data)
     
    one_player_dataset = pd.json_normalize(player_data_list)
    return one_player_dataset

In [11]:
gather_all_player_match_data(puuid, match_ids, mass_region, api_key)

EUW1_6639695853
EUW1_6639674317
EUW1_6639653949
EUW1_6639645836
EUW1_6639597933
EUW1_6639585296
EUW1_6639577072
EUW1_6636841353
EUW1_6636834892
EUW1_6636812567


,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,challenges.mythicItemUsed,challenges.baronBuffGoldAdvantageOverThreshold,challenges.earliestBaron,challenges.highestCrowdControlScore,challenges.teleportTakedowns,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.soloTurretsLategame,challenges.thirdInhibitorDestroyedTime,challenges.shortestTimeToAceFromFirstTakedown,challenges.highestChampionDamage
0,0,0,7,0,0,0,0,11162,13,268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,2,0,0,0,0,8448,11,268,...,4644.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,5,0,0,0,0,13129,15,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,1,5388,9,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,7,0,0,0,3,18134,17,69,...,6656.0,1.0,1262.538576,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,0,0,5,0,0,0,1,16593,17,24,...,6632.0,NaN,1562.990415,NaN,1.0,0.334955,1.0,1414.100635,NaN,NaN
6,0,0,7,0,0,0,0,19794,18,21,...,NaN,1.0,1539.039600,NaN,NaN,0.643939,2.0,NaN,NaN,NaN
7,0,0,5,0,0,0,2,12774,14,51,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
8,0,0,15,0,0,0,0,18373,18,950,...,6692.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.880655,NaN
9,0,0,8,0,0,0,0,19299,18,112,...,6653.0,NaN,1441.632782,NaN,1.0,NaN,NaN,NaN,19.313266,1.0


In [12]:
def master_function(summoner_name, region, mass_region, no_games, queue_id, api_key):
    puuid = get_puuid(region, summoner_name, api_key)
    match_ids = get_match_ids(puuid, mass_region, no_games, queue_id, api_key)
    one_player_dataset = gather_all_player_match_data(puuid, match_ids, mass_region, api_key)
    return one_player_dataset

In [13]:
master_function(summoner_name, region, mass_region, no_games, queue_id, api_key)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/fallinglove?api_key=RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02
https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/-3doM1YXNXH8dExYK3kq8vw06KShdkJ-hECzwo8seV1lkHujhDq2R1kzDggD6ZZHG6Xbs8URshdTaQ/ids?start=0&count=10&queue=420&api_key=RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02
EUW1_6639695853
EUW1_6639674317
EUW1_6639653949
EUW1_6639645836
EUW1_6639597933
EUW1_6639585296
EUW1_6639577072
EUW1_6636841353
EUW1_6636834892
EUW1_6636812567


,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,challenges.mythicItemUsed,challenges.baronBuffGoldAdvantageOverThreshold,challenges.earliestBaron,challenges.highestCrowdControlScore,challenges.teleportTakedowns,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.soloTurretsLategame,challenges.thirdInhibitorDestroyedTime,challenges.shortestTimeToAceFromFirstTakedown,challenges.highestChampionDamage
0,0,0,7,0,0,0,0,11162,13,268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,2,0,0,0,0,8448,11,268,...,4644.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,5,0,0,0,0,13129,15,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,1,5388,9,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,7,0,0,0,3,18134,17,69,...,6656.0,1.0,1262.538576,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,0,0,5,0,0,0,1,16593,17,24,...,6632.0,NaN,1562.990415,NaN,1.0,0.334955,1.0,1414.100635,NaN,NaN
6,0,0,7,0,0,0,0,19794,18,21,...,NaN,1.0,1539.039600,NaN,NaN,0.643939,2.0,NaN,NaN,NaN
7,0,0,5,0,0,0,2,12774,14,51,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
8,0,0,15,0,0,0,0,18373,18,950,...,6692.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.880655,NaN
9,0,0,8,0,0,0,0,19299,18,112,...,6653.0,NaN,1441.632782,NaN,1.0,NaN,NaN,NaN,19.313266,1.0


In [14]:
api_key = "RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02"
region = "euw1"
summoner_name = "fallinglove"
mass_region = 'europe'
no_games = 100
queue_id = 420
df_fallinglove = master_function(summoner_name, region, mass_region, no_games, queue_id, api_key)
df_fallinglove

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/fallinglove?api_key=RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02
https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/-3doM1YXNXH8dExYK3kq8vw06KShdkJ-hECzwo8seV1lkHujhDq2R1kzDggD6ZZHG6Xbs8URshdTaQ/ids?start=0&count=100&queue=420&api_key=RGAPI-098cd71e-5e71-45e2-acbe-0f0230817f02
EUW1_6639695853
EUW1_6639674317
EUW1_6639653949
EUW1_6639645836
EUW1_6639597933
EUW1_6639585296
EUW1_6639577072
EUW1_6636841353
EUW1_6636834892
EUW1_6636812567
EUW1_6636783978
EUW1_6635557299
EUW1_6635562500
EUW1_6635547085
EUW1_6635541758
EUW1_6635532830
EUW1_6635153681
EUW1_6635075202
EUW1_6632617176
EUW1_6632614530
EUW1_6632583404
EUW1_6632225266
EUW1_6632161422
EUW1_6632095094
EUW1_6631064178
EUW1_6631051677
EUW1_6630481281
EUW1_6627015296
EUW1_6627007229
EUW1_6626999920
EUW1_6626900770
EUW1_6625807295
EUW1_6625800330
EUW1_6625651653
EUW1_6624462132
EUW1_6624423993
EUW1_6624376539
EUW1_6624320719
EUW1_6623176340
EUW1_6623169155
EUW1_66

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,challenges.thirdInhibitorDestroyedTime,challenges.shortestTimeToAceFromFirstTakedown,challenges.highestChampionDamage,challenges.fastestLegendary,challenges.hadAfkTeammate,challenges.fasterSupportQuestCompletion,challenges.junglerKillsEarlyJungle,challenges.killsOnLanersEarlyJungleAsJungler,challenges.highestWardKills,challenges.earliestElderDragon
0,0,0,7,0,0,0,0,11162,13,268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,2,0,0,0,0,8448,11,268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,5,0,0,0,0,13129,15,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,1,5388,9,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,7,0,0,0,3,18134,17,69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,4,1,0,0,0,24784,18,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,0,0,2,0,0,0,0,12525,14,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,0,0,3,0,0,0,1,14732,16,75,...,NaN,36.338982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,0,0,4,0,0,0,0,16682,17,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_fallinglove.to_csv('fallinglove.csv', index=False)